In [6]:
import random
import time

class Board:
    def __init__(self, size):
        self.size = size
        self.board = [[random.choice([0, 1]) for _ in range(size)] for _ in range(size)]  # Initialize board with random coins
        self.board[0][0] = 0
        self.board[size-1][size-1] = 0
        self.players = []

    def update(self, player1_pos, player2_pos):
        self.players = [player1_pos, player2_pos]  # Update players' positions

    def is_over(self):
        # Check if the game is over (no more coins on the board)
        return sum(sum(row) for row in self.board) == 0

    def print_state(self, player1_pos, player2_pos, player1_score, player2_score):
        # Print the state of the game (board, player positions, scores)
        print("Current state of the game:")
        for row in self.board:
            print(" ".join(map(str, row)))
        print(f"Player 1 position: {player1_pos}, score: {player1_score}")
        print(f"Player 2 position: {player2_pos}, score: {player2_score}")
        print()

class Player:
    def __init__(self, name, position):
        self.name = name
        self.score = 0
        self.position = position

    def move(self, direction, board, change=False):
        # Move the player in the specified direction
        new_position = self.position.copy()  # Create a copy of the current position
        if direction == "up" and new_position[1] > 0:
            new_position[1] -= 1
        elif direction == "down" and new_position[1] < board.size - 1:
            new_position[1] += 1
        elif direction == "left" and new_position[0] > 0:
            new_position[0] -= 1
        elif direction == "right" and new_position[0] < board.size - 1:
            new_position[0] += 1
        
        # Update the player's position
        if change:
            self.position = new_position
        
        return new_position

def minimax(board, player, opponent, depth, maximizing_player, alpha, beta):
    if depth == 0 or board.is_over():
        # print("reach 0 score =",player.score - opponent.score)
        return player.score - opponent.score  # Evaluation function

    if maximizing_player:
        max_eval = float("-inf")
        for move in ["up", "down", "left", "right"]:
            new_position = player.move(move, board)
            # add the score counter here with the condition of new_position containing coin
            if new_position != player.position:  # Exclude staying still as an option
                eval = minimax(board, Player(player.name, new_position), opponent, depth - 1, False, alpha, beta)
            else:
                eval = float("-inf") if maximizing_player else float("inf")  # Staying still is not allowed
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = float("inf")
        for move in ["up", "down", "left", "right"]:
            new_position = opponent.move(move, board)
            if new_position != opponent.position:  # Exclude staying still as an option
                eval = minimax(board, player, Player(opponent.name, new_position), depth - 1, True, alpha, beta)
            else:
                eval = float("-inf") if maximizing_player else float("inf")  # Staying still is not allowed
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval


def opposite(direction):
    opposites = {"up": "down", "down": "up", "left": "right", "right": "left"}
    return opposites[direction]

# Main game loop
def play_game(size=8, depth=4):
    board = Board(size)
    player1_position = [0, 0]
    player2_position = [size - 1, size - 1]
    player1 = Player("Player 1", player1_position)
    player2 = Player("Player 2", player2_position)
    board.update(player1_position, player2_position)  # Initialize player positions

    # Print initial state of the game
    print("Initial state of the game:")

    while not board.is_over():
        # Print state of the game
        board.print_state(player1.position, player2.position, player1.score, player2.score)
        time.sleep(1)  # Add a delay of 1 second

        # Player 1's turn
        max_eval = float("-inf")
        best_move = None
        possible_move = None
        for move in ["up", "down", "left", "right"]:
            new_position = player1.move(move, board)
            print(move,player1.position,new_position)
            if new_position == player1.position:
                continue
            eval = minimax(board, Player(player1.name, new_position), player2, depth - 1, False, float("-inf"), float("inf"))
            if eval > max_eval:
                max_eval = eval
                best_move = move
            if eval not in [float('inf'), float('-inf')]:  # Check if the move is valid
                possible_move = move
            print(f"{move},{eval}")
        if best_move == None:
            best_move = possible_move
        print(f"Player 1 moves : {best_move}")
        # Update player1's position
        player1.move(best_move, board, True)
        # Update player1's score if there is a coin at the new position
        if board.board[player1.position[1]][player1.position[0]] == 1:
            player1.score += 1
            board.board[player1.position[1]][player1.position[0]] = 0  # Remove the coin from the board


        # Player 2's turn
        max_eval = float("-inf")
        best_move = None
        possible_move = None
        for move in ["up", "down", "left", "right"]:
            new_position = player2.move(move, board)
            print(move,player2.position,new_position)
            if new_position == player2.position:
                continue
            eval = minimax(board, player1, Player(player2.name, new_position), depth - 1, True, float("-inf"), float("inf"))
            if eval > max_eval:
                max_eval = eval
                best_move = move
            if eval not in [float('inf'), float('-inf')]:  # Check if the move is valid
                possible_move = move
            print(f"{move},{eval}")
        if best_move == None:
            best_move = possible_move
        print(f"Player 2 moves : {best_move}")
        # Update player2's position
        player2.move(best_move, board, True)
        # Update player2's score if there is a coin at the new position
        if board.board[player2.position[1]][player2.position[0]] == 1:
            player2.score += 1
            board.board[player2.position[1]][player2.position[0]] = 0  # Remove the coin from the board


    # Print final state of the game
    board.print_state(player1.position, player2.position, player1.score, player2.score)

    # Print final scores
    print("Final scores:")
    print(f"{player1.name}: {player1.score}")
    print(f"{player2.name}: {player2.score}")

# Start the game
play_game()

Initial state of the game:
Current state of the game:
0 0 1 1 0 0 1 1
1 1 1 1 1 0 0 0
0 1 1 1 0 1 1 1
0 1 0 0 0 1 1 1
1 1 0 1 1 1 1 0
1 0 1 0 1 0 1 1
1 1 0 1 0 0 1 1
1 1 1 0 1 0 0 0
Player 1 position: [0, 0], score: 0
Player 2 position: [7, 7], score: 0

up [0, 0] [0, 0]
down [0, 0] [0, 1]
down,0
left [0, 0] [0, 0]
right [0, 0] [1, 0]
right,0
Player 1 moves : down
up [7, 7] [7, 6]
up,0
down [7, 7] [7, 7]
left [7, 7] [6, 7]
left,0
right [7, 7] [7, 7]
Player 2 moves : up
Current state of the game:
0 0 1 1 0 0 1 1
0 1 1 1 1 0 0 0
0 1 1 1 0 1 1 1
0 1 0 0 0 1 1 1
1 1 0 1 1 1 1 0
1 0 1 0 1 0 1 1
1 1 0 1 0 0 1 0
1 1 1 0 1 0 0 0
Player 1 position: [0, 1], score: 1
Player 2 position: [7, 6], score: 1

up [0, 1] [0, 0]
up,0
down [0, 1] [0, 2]
down,0
left [0, 1] [0, 1]
right [0, 1] [1, 1]
right,0
Player 1 moves : up
up [7, 6] [7, 5]
up,0
down [7, 6] [7, 7]
down,0
left [7, 6] [6, 6]
left,0
right [7, 6] [7, 6]
Player 2 moves : up
Current state of the game:
0 0 1 1 0 0 1 1
0 1 1 1 1 0 0 0
0 1 1 1 0 

KeyboardInterrupt: 

In [ ]:
MOVES = ["u","d","l","r"]
FLOOR = float("-inf")
CEIL = float("inf")

In [ ]:
class PacmanAI:

    def __init__(self, board, x=0, y=0, xe=7, ye=7, moves="", score=0, scoree=0, size=8) -> None:
        self.board = board
        self.size = size
        self.x = x
        self.y = y
        self.xe = xe
        self.ye = ye
        self.moves = moves
        self.score = score
        self.scoree = scoree

    def __str__(self):
        out = "Board:\n"
        for i in range(self.size):
            out += str(self.board[i*self.size:(i+1)*self.size])
            out += "\n"
        out += f"Position = ({self.x},{self.y})\n"
        out += f"EPosition = ({self.xe},{self.ye})\n"
        out += f"Moves = {self.moves}\n"
        out += f"Score = {self.score}\n"
        return out
    
    def send_pos(self):
        return list([self.x,self.y,self.score,self.board.copy()])
    
    def update_enemy(self, new):
        self.xe = new[0]
        self.ye = new[1]
        self.scoree = new[2]
        self.board = new[3]
        return

    def is_over(self):
        return sum(self.board) == 0
    
    def gen_child(self):
        l = list()
        for move in MOVES:
            x = self.x
            y = self.y
            score = self.score
            if move == "u":
                y = max(self.y-1,0)
            elif move == "d":
                y = min(self.y+1,self.size-1)
            elif move == "l":
                x = max(self.x-1,0)
            elif move == "r":
                x = min(self.x+1,self.size-1)
            board = self.board.copy()
            dest_score = board[y*self.size + x]
            if dest_score != 0:
                score += dest_score
                board[y*self.size + x] = 0
            child = PacmanAI(board,x,y,self.xe,self.ye,self.moves+move,score,self.scoree,self.size)
            l.append(child)
        return l
    
    def scorer(self):
        return self.score - self.scoree

    def minimax(self, maximizer):

        if self.is_over():
            return list(self.scorer(),self.moves)
        children = self.gen_child()

        if maximizer:
            max_score = FLOOR
            max_move = ""
            for child in children:
                # if self.is_same(child):
                #     pass
                minimax_score, minimax_move = child.minimax(False)
                if minimax_score > max_score:
                    max_score = minimax_score
                    max_move = minimax_move
            return list(max_score, max_move)     
            
        else:
            min_score = CEIL
            min_move = ""   
            for child in children:
                # if self.is_same(child):
                #     pass
                minimax_score, minimax_move = child.minimax(True)
                if minimax_score < min_score:
                    min_score = minimax_score
                    min_move = minimax_move 
            return list(min_score, min_move)    
        

In [ ]:
l = [0 for i in range(16)]
l[1] = 1

In [ ]:
p = PacmanAI(l,size=4)

In [ ]:
c = p.gen_child()

In [ ]:
for child in c:
    print(child)

Board:
[0, 1, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
Position = (0,0)
EPosition = (7,7)
Moves = u
Score = 0

Board:
[0, 1, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
Position = (0,1)
EPosition = (7,7)
Moves = d
Score = 0

Board:
[0, 1, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
Position = (0,0)
EPosition = (7,7)
Moves = l
Score = 0

Board:
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
Position = (1,0)
EPosition = (7,7)
Moves = r
Score = 1



In [ ]:
m = p.minimax(True)